# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree
import time

# Importing and Splitting data

In [2]:
modelEncoder = LabelEncoder()
transmissionEncoder = LabelEncoder()
fuelTypeEncoder = LabelEncoder()

def dataset(brand):

    file = pd.read_csv(brand, quotechar='"', skipinitialspace=True)

    for i in ['year']:
        q75,q25 = np.percentile(file.loc[:,i],[75,25])
        IQR = q75-q25
    
        max = q75+(1.5*IQR)
        min = q25-(1.5*IQR)
    
        file.loc[file[i] < min, i] = np.nan
        file.loc[file[i] > max, i] = np.nan

    file = file.dropna(axis = 0)

    modelEncoder.fit(file["model"])
    file["model"] = modelEncoder.transform(file["model"])

    transmissionEncoder.fit(file["transmission"])
    file["transmission"] = transmissionEncoder.transform(file["transmission"])

    fuelTypeEncoder.fit(file["fuelType"])
    file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

    file = file.head(10000)
    # X = file.drop(columns = ['price'])
    # Y = file.price
    # print(file)

    X = file.drop(['price'], axis = 1).to_numpy()
    # X = file[['year', 'mileage', 'tax', 'mpg', 'engineSize']].to_numpy(dtype = 'int')
    Y = file['price'].values.reshape(-1,1)

#     print(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)
    
    return  X_train, X_test, Y_train, Y_test, file

X_train, X_test, Y_train, Y_test, file= dataset("../UKUsedCarDataSet/vw.csv") # Use Audi dataset as default for KNN analysis

# Using Scikit-Learn

myTree = DecisionTreeRegressor(random_state = 601, max_depth = 34, min_samples_split = 3) #, max_depth = 3
myTree.fit(X_train, Y_train)
print("R Squared: ", myTree.score(X_test, Y_test))

# GridSeacrhCV to find optimal parameters

from sklearn.model_selection import GridSearchCV

params = {'min_samples_split': list(range(2, 100)), 'max_leaf_nodes': list(range(2, 100)), 'max_depth': list(range(2, 100))}
grid_search_cv = GridSearchCV(myTree, params, cv=3)
grid_search_cv.fit(X_train, Y_train)

grid_search_cv.best_estimator_

# Visualising Decsion Tree:

X1 = file.drop(['price'], axis = 1)

plt.figure(figsize=(20, 10), dpi = 200)
plot_tree(myTree, feature_names = X1.columns, filled=True)

# RMSE

In [3]:
def rmse(h, y):
  return sqrt(mean_squared_error(h, y))

# Node Class

In [4]:
# Node class to initialise instances of each 
class Node():
    
    def __init__(self, feature = None, limit = None, leftSide = None, rightSide = None, gain = None, leaf = None):
        
        self.feature = feature
        self.limit = limit
        self.leftSide = leftSide
        self.rightSide = rightSide
        self.gain = gain
        self.leaf = leaf 

# Decision Tree Regression Class

In [5]:
class decisionTree():
    def __init__(self, minSamples, maxDepth):
        self.root = None
        self.minSamples = minSamples
        self.maxDepth = maxDepth
        
    def infoGain(self, parent, leftNode, rightNode):
        leftWeight = len(leftNode) / len(parent)
        rightWeight = len(rightNode) / len(parent)
        
        informationGain = np.var(parent) - (leftWeight * np.var(leftNode) + rightWeight * np.var(rightNode))

        return informationGain
        
    def splitTree(self, trainingSet, feature, limit):
        leftBranch = []
        rightBranch = []
        for i in trainingSet:
            if i[feature] <= limit:
                leftBranch.append(i)
            else:
                rightBranch.append(i)
        rightBranch = np.array(rightBranch)
        leftBranch = np.array(leftBranch)
        return leftBranch, rightBranch
        
    def bestSplit(self, trainingSet, X):
        bestSplitt = {} 
        biggestGain = -1
        for feature in range(X.shape[1]): 
            featureValues = []
            for i in range(len(trainingSet)):
                featureValues.append(trainingSet[i, feature])
            thresholds = np.unique(featureValues)
            for j in thresholds:
                leftSide, rightSide = self.splitTree(trainingSet, feature, j) #splits node into 2 sub-trees
                if (len(leftSide) > 0 and len(rightSide) > 0 ):
                    parent = []
                    for i in range(len(trainingSet)):
                        parent.append(trainingSet[i, -1])

                    leftNode = []
                    for i in range(len(leftSide)):
                        leftNode.append(leftSide[i, -1])
                        
                    rightNode = []
                    for i in range(len(rightSide)):
                        rightNode.append(rightSide[i, -1])

                    currentGain = self.infoGain(parent, leftNode, rightNode) 
                    if currentGain > biggestGain:
                        
                        bestSplitt["feature"] = feature
                        bestSplitt["limit"] = j
                        bestSplitt["leftSide"] = leftSide
                        bestSplitt["rightSide"] = rightSide
                        bestSplitt["gain"] = currentGain
                        biggestGain = currentGain
                
        return bestSplitt
   
        
    def treeBuild(self, trainingSet, currentDepth = 0):
        
#       #Split training into features and labels
        X = trainingSet[:,:-1] # everything but the last value
        Y = []
        for i in range(len(trainingSet)):
            Y.append(trainingSet[i, -1]) # only the last value
        
        #iterates until this condition is met
        if X.shape[0] >= self.minSamples and currentDepth <= self.maxDepth:
            bestSplitNode = self.bestSplit(trainingSet, X)
            
            if "gain" in bestSplitNode and bestSplitNode["gain"] > 0:
                leftTree = self.treeBuild(bestSplitNode["leftSide"], currentDepth + 1)
                rightTree = self.treeBuild(bestSplitNode["rightSide"], currentDepth + 1)
                node = Node(bestSplitNode["feature"], bestSplitNode["limit"], leftTree, rightTree, bestSplitNode["gain"])

                return node
        
        leafValue = np.mean(Y) #calculates mean of leaf nodes
        val = Node(leaf = leafValue)
        return val
    
    def predictionLoop(self, testRow, root):
        if root.leaf != None: #not empty
            return root.leaf
        
        featureVal = testRow[root.feature]
        if featureVal <= root.limit:
            return self.predictionLoop(testRow, root.leftSide)
        else:
            return self.predictionLoop(testRow, root.rightSide)
        
   
    def predict(self, xTest):
        predictions = []
        for row in xTest:
            predictions.append(self.predictionLoop(row, self.root)) 
        return predictions

        
        
    def fit(self, X, Y):
        trainingSet = np.concatenate((X, Y), axis=1) #Joins training data back together
        self.root = self.treeBuild(trainingSet)

# Training the algorithm

In [6]:
timer = time.time()
myTree = decisionTree(3, 34) # minSamples = 3, maxDepth = 34
myTree.fit(X_train, Y_train) # Trains the model using the training set

# Evaluation

In [7]:
def evaluation(X_test, Y_test):    
    Y_pred = myTree.predict(X_test)
    error = rmse(Y_test, Y_pred) 
    print('The RMSE value is:', round(error, 2))

    # for i in range(len(y_pred)):
    #     print("\nOriginal value:", yTest[i], "vs the predicted value:", y_pred[i])
    #     print("The difference is:", yTest[i] - y_pred[i])

    mean= np.mean(file["price"])

    print("R VALUE:", 1 - (error/mean))

evaluation(X_test, Y_test)
print("\n ***Predicted in", time.time() - timer,"seconds***")

#rmse without outliers: 3027.4338424903326

The RMSE value is: 1672.16
R VALUE: 0.8932134432466091

 ***Predicted in 153.87267208099365 seconds***


# User Input Predictions

inputPred = []
entries = []

def userInput():
    chooseBrand = input("Choose your car brand: Audi, BMW, Ford, Hyundai, Mercedes, Skoda, Toyota, Vauxhall or Volkswagen \n")
    
    match chooseBrand:
        case "Audi":
            return "../UKUsedCarDataSet/audi.csv"
        case "BMW":
            return "../UKUsedCarDataSet/bmw.csv"
        case "Ford":
            return "../UKUsedCarDataSet/ford.csv"
        case "Hyundai":
            return "../UKUsedCarDataSet/hyundi.csv"
        case "Mercedes":
            return "../UKUsedCarDataSet/merc.csv"
        case "Skoda":
            return "../UKUsedCarDataSet/skoda.csv"
        case "Toyota":
            return "../UKUsedCarDataSet/toyota.csv"
        case "Vauxhall":
            return "../UKUsedCarDataSet/vauxhall.csv"
        case "Volkswagen":
            return "../UKUsedCarDataSet/volkswagen.csv"
        case _:
            print("Invalid input")
            return userInput()

X_train, X_test, Y_train, Y_test, file = dataset(userInput())

print("\n ***Training Tree Model***")
timer = time.time()

myTree = decisionTree(3, 93)  
myTree.fit(X_train, Y_train)

print("\n List of models:")
print(list(modelEncoder.classes_))

inputPred.append((modelEncoder.transform([input("\nWhat Model is your car? ")]))[0])
inputPred.append(int(input("What year is your car? ")))
inputPred.append((transmissionEncoder.transform([input("What transmission is your car? ")]))[0])
inputPred.append(int(input("How much mileage does your car have? ")))
inputPred.append((fuelTypeEncoder.transform([input("What's your car fuel type? ")]))[0])
inputPred.append(int(input("How much is your cars tax? ")))
inputPred.append(float(input("What's MPG of your car? ")))
inputPred.append(float(input("What the engine size of your car? ")))
entries.append(inputPred)


print("\n ***Predicting***")
Y_pred = myTree.predict([inputPred])
print("\n Predicted price for your car is: £", round(Y_pred[0], 2))
print("\n ***Predicted in", time.time() - timer,"seconds***")

# RS6,2016,Semi-Auto,49050,Petrol,325,29.4,4.0 -- Price is £44,985   old Pred = £41,233.30,  new pred = £45492.50
# BMW,5 Series,2019,Semi-Auto,4405,Petrol,145,48.7,2.0     Price = £26,000 old Pred = £27,077.49, new pred = £26500.00